<center><h1>Using Word2Vec and an RNN</h1></center>

In [1]:
import numpy as np
import pandas as pd

from gensim.models import word2vec, Word2Vec
import logging

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

import tensorflow as tf
from tensorflow.python.ops.rnn_cell import GRUCell, DropoutWrapper, MultiRNNCell
from tensorflow.python.ops.nn import relu, elu, softmax
from tensorflow.contrib.layers import fully_connected
from tensorflow.python.framework.ops import reset_default_graph

import re
import nltk
from nltk.corpus import stopwords
from bs4 import BeautifulSoup

/home/boniface/.local/lib/python3.5/site-packages/gensim/utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


In [2]:
lab = pd.read_csv( "labeledTrainData.tsv", header=0, delimiter="\t", quoting=3 )
test = pd.read_csv( "testData.tsv", header=0, delimiter="\t", quoting=3 )
unlab = pd.read_csv( "unlabeledTrainData.tsv", header=0, delimiter="\t", quoting=3 )

## Cleaning the dataset and processing it to use Word2Vec

In [ ]:
def review_to_wordlist(raw_review):
    """Function to convert a raw review to a string of words
    The input is a single string (a raw movie review), and 
    the output is a single string (a preprocessed movie review)
    """
    # 1. Remove HTML
    review_text = BeautifulSoup(raw_review).get_text() 
    #
    # 2. Remove non-letter characters        
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    #
    # 3. Convert to lower case, split into individual words
    words = letters_only.lower().split()                             
    #
    # 4.Return the result.
    return words

tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

def review_to_sentences( review, tokenizer):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append(review_to_wordlist(raw_sentence))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

In [ ]:
sentences = []

print ("Parsing sentences from training set")
for review in lab.review:
    sentences += review_to_sentences(review, tokenizer)

print("Parsing sentences from unlabeled set")
for review in unlab.review:
    sentences += review_to_sentences(review, tokenizer)

Parsing sentences from training set


/home/boniface/.local/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file /home/boniface/anaconda3/envs/tensorflow/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "html.parser")

  markup_type=markup_type))
/home/boniface/.local/lib/python3.5/site-packages/bs4/__init__.py:219: UserWarning: "b'.'" looks like a filename, not markup. You shouldprobably open this file and pass the filehandle intoBeautiful Soup.
  'Beautiful Soup.' % markup)
/home/boniface/.local/lib/python3.5/site-packages/bs4/__init__.py:282: UserWarning: "http://www.h

In [ ]:
len(sentences)

## Training the Word2Vec model

In [ ]:
%%time

# sert à sortir de jolis messages de temps en temps
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

num_features = 300    # Word vector dimensionality
min_word_count = 40   # Minimum word count
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window sized
downsampling = 1e-3   # Downsample setting for frequent words

print("Training model...")
model = word2vec.Word2Vec(sentences, workers=num_workers, size=num_features, min_count = min_word_count, \
                          window = context, sample = downsampling)

model.init_sims(replace=True)

model_name = "300features_40minwords_10context"
model.save(model_name)

In [ ]:
model.doesnt_match('man woman elderly children spoon'.split())

In [32]:
model.most_similar('man')

[('woman', 0.6198972463607788),
 ('lady', 0.5801385045051575),
 ('lad', 0.5663866996765137),
 ('monk', 0.5272423624992371),
 ('farmer', 0.5249172449111938),
 ('men', 0.5218124985694885),
 ('millionaire', 0.5207440853118896),
 ('soldier', 0.517442524433136),
 ('guy', 0.5158977508544922),
 ('person', 0.5004771947860718)]

In [33]:
model.most_similar('france')

[('spain', 0.8204919695854187),
 ('italy', 0.8125512003898621),
 ('germany', 0.8049379587173462),
 ('england', 0.7958934307098389),
 ('europe', 0.790851354598999),
 ('india', 0.7580022811889648),
 ('greece', 0.7553827166557312),
 ('poland', 0.7497527599334717),
 ('austria', 0.7405729293823242),
 ('northern', 0.7390496134757996)]

In [15]:
model.most_similar(positive=['woman', 'king'], negative=['man'])

[('princess', 0.5144521594047546),
 ('queen', 0.5030089616775513),
 ('prince', 0.4746342897415161),
 ('elizabeth', 0.45409542322158813),
 ('rudolf', 0.451641321182251),
 ('bride', 0.4308673143386841),
 ('juliet', 0.42891353368759155),
 ('lion', 0.42094939947128296),
 ('stella', 0.41837969422340393),
 ('victoria', 0.4146151542663574)]

In [32]:
model.similarity('running', 'run')

0.59717562174767336

In [33]:
model.similarity('running', 'walking')

0.63969969394729009

## Using the embeddings to get the features

In [3]:
model = Word2Vec.load('300features_40minwords_10context') # load the saved model

In [4]:
model.syn0.shape # numpy array containing the embeddings number of words x embedding dimensionality

(16490, 300)

In [5]:
stops = set(stopwords.words("english")) # searching a set is much faster than a list

def review_to_words(raw_review):
    """Function to convert a raw review to a string of words
    The input is a single string (a raw movie review), and 
    the output is a single string (a preprocessed movie review)
    """
    # 1. Remove HTML
    review_text = BeautifulSoup(raw_review).get_text() 
    #
    # 2. Remove non-letters        
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    #
    # 3. Convert to lower case, split into individual words
    words = letters_only.lower().split()                             
    #
    # 4. Remove stop words
    meaningful_words = [w for w in words if not w in stops]
    #
    # 5. Return the result.
    return meaningful_words

def clean_dataset(dataset):
    """Loops the previous function to clean an entire dataset"""
        
    new = dataset
    revs = []
    for rev in new.review:
        revs.append(review_to_words(rev))
        
    new.review = revs 
    return new

In [6]:
data = clean_dataset(lab)

/home/boniface/.local/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file /home/boniface/anaconda3/envs/tensorflow/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "html.parser")

  markup_type=markup_type))


In [7]:
train, val = train_test_split(data, train_size=0.8, test_size=0.2, random_state=42)
x_train, y_train = train.review, train.sentiment
x_val, y_val = val.review, val.sentiment

In [8]:
x_train, y_train = np.array(x_train), np.array(y_train)
x_val, y_val = np.array(x_val), np.array(y_val)

In [9]:
def embed_reviews(model, reviews):
    """Replaces each word by its embedding."""
    all_reviews = []
    for rev in reviews:
        embed = []
        for word in rev:
            if word in model.vocab.keys(): # only words that appear at least 40 times have an embedding
                embed.append(model[word])
            else:
                embed.append(model['the']) # I tried to take the least significant word but not sure if it's a good idea
        
        embed = np.array(embed)
        all_reviews.append(embed)
        
    return np.array(all_reviews)

In [10]:
x_train = embed_reviews(model, x_train)
x_val = embed_reviews(model, x_val)

## Building the model

In [11]:
def pad(features, batch_size):
    """Pads the features with zeros, so that we can have a tensor."""
    lmax = 0
    for line in features:
        if line.shape[0] > lmax:
            lmax = line.shape[0]

    new = np.zeros((batch_size, lmax, 300))
    for i,sample in enumerate(features):
        for j,word in enumerate(sample):
            for k,coeff in enumerate(word):
                new[i,j,k] = coeff
                
    return new

def onehot(t, n_classes):
    """Does the one-hot encoding"""
    out = np.zeros((t.shape[0], n_classes))
    for row, col in enumerate(t):
        out[row, col] = 1
    return out

In [12]:
n_classes = 2
#n_layers = 2
n_hidden = 128
state_size = 64

reset_default_graph()

x_pl = tf.placeholder(tf.float32, [None, None, 300])

def length(sequence):
    used = tf.sign(tf.reduce_max(tf.abs(sequence), reduction_indices=2)) # put a 1 where there is a non-zero value
    length = tf.reduce_sum(used, reduction_indices=1) # sums the ones to obtain the actual sequence length
    length = tf.cast(length, tf.int32)
    return length

cell = GRUCell(state_size)
#cell = DropoutWrapper(cell, output_keep_prob=dropout)
#cell = MultiRNNCell([cell] * n_layers) # in order to take into account different time steps

outputs, state = tf.nn.dynamic_rnn(cell, x_pl, sequence_length=length(x_pl), dtype=tf.float32, scope='unfold')
# when the argument sequence_length is given, state is the last RELEVANT state

fc = fully_connected(state, n_hidden, activation_fn=elu, scope='fc')
y = fully_connected(fc, n_classes, activation_fn=softmax, scope='softmax')

In [13]:
epsilon = 1e-8
learning_rate = 1e-4

y_pl = tf.placeholder(tf.float32, [None, n_classes])

cross_entropy = -tf.reduce_sum(y_pl * tf.log(y+epsilon))
cross_entropy = tf.reduce_mean(cross_entropy)

optimizer = tf.train.AdamOptimizer(learning_rate)
# maybe we should bring the gradient norm to 1
train_op = optimizer.minimize(cross_entropy)

correct_predictions = tf.equal(tf.argmax(y, 1), tf.argmax(y_pl, 1))
accuracy = tf.reduce_mean(tf.cast(correct_predictions, tf.float32))
auc = tf.contrib.metrics.streaming_auc(y, y_pl)

## Training and evaluating our model

In [14]:
batch_size = 100
n_epochs = 10

n_batches_train = len(x_train) // batch_size
n_batches_val = len(x_val) // batch_size

train_acc, train_auc, train_loss = [], [], []
val_acc, val_auc, val_loss = [], [], []

In [15]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
sess.run(tf.local_variables_initializer()) # needed for the auc score

In [18]:
for epoch in range(n_epochs):
    
    for i in range(n_batches_train):
        idx = np.arange(i*batch_size, (i+1)*batch_size)
        x_batch = pad(x_train[idx], batch_size) # we do the padding per batch for memory efficience
        y_batch = onehot(y_train[idx], n_classes)
        fetches_train = [train_op]
        feed_train = {x_pl: x_batch, y_pl: y_batch}
        sess.run(fetches_train, feed_train)
    
    cur_loss = 0
    cur_acc = 0
    cur_auc = 0
    for i in range(n_batches_train):
        idx = np.arange(i*batch_size, (i+1)*batch_size)
        x_batch = pad(x_train[idx], batch_size)
        y_batch = onehot(y_train[idx], n_classes)
        fetches_eval_train = [cross_entropy, accuracy, auc]
        feed_eval_train = {x_pl: x_batch, y_pl: y_batch}
        batch_loss, batch_acc, batch_auc = sess.run(fetches_eval_train, feed_eval_train)
        batch_auc, _ = batch_auc # it's a tupple that contains twice the same value
        cur_loss += batch_loss
        cur_acc += batch_acc
        cur_auc += batch_auc
        
    train_loss.append(cur_loss/n_batches_train)
    train_acc.append(cur_acc/n_batches_train)
    train_auc.append(cur_auc/n_batches_train)
    
    cur_loss = 0
    cur_acc = 0
    cur_auc = 0
    for i in range(n_batches_val):
        idx = np.arange(i*batch_size, (i+1)*batch_size)
        x_batch = pad(x_val[idx], batch_size)
        y_batch = onehot(y_val[idx], n_classes)
        fetches_eval_val = [cross_entropy, accuracy, auc]
        feed_eval_val = {x_pl: x_batch, y_pl: y_batch}
        batch_loss, batch_acc, batch_auc = sess.run(fetches_eval_val, feed_eval_val)
        batch_auc, _ = batch_auc
        cur_loss += batch_loss
        cur_acc += batch_acc
        cur_auc += batch_auc
            
    val_loss.append(cur_loss/n_batches_val)
    val_acc.append(cur_acc/n_batches_val)
    val_auc.append(cur_auc/n_batches_val)
    
    print('Mean batch loss train:', train_loss[-1], 'Mean batch loss valid:', val_loss[-1])
    print('Train accuracy:', train_acc[-1], 'Valid accuracy:', val_acc[-1])
    print('Train ROC auc score:', train_auc[-1], 'Valid ROC auc score:', val_auc[-1])
    
    if len(val_auc) > 1 and val_auc[-1] < val_auc[-2]:
        print('Early stopping')
        break

Mean batch loss train: 30.3162347317 Mean batch loss valid: 29.7588188934
Train accuracy: 0.875450002253 Valid accuracy: 0.871799999475
Train ROC auc score: 0.936852068603 Valid ROC auc score: 0.937307502031
Mean batch loss train: 30.1046811581 Mean batch loss valid: 29.5837371445
Train accuracy: 0.876150001585 Valid accuracy: 0.871799998283
Train ROC auc score: 0.937853122056 Valid ROC auc score: 0.938240491152
Mean batch loss train: 29.9094496632 Mean batch loss valid: 29.4240163803
Train accuracy: 0.87650000155 Valid accuracy: 0.872799999714
Train ROC auc score: 0.938713358641 Valid ROC auc score: 0.939048223495
Mean batch loss train: 29.7430462837 Mean batch loss valid: 29.2907396698
Train accuracy: 0.87715000093 Valid accuracy: 0.87439999938
Train ROC auc score: 0.939463941157 Valid ROC auc score: 0.939758183956
Mean batch loss train: 29.5887020111 Mean batch loss valid: 29.168461113
Train accuracy: 0.877300000787 Valid accuracy: 0.877199997902
Train ROC auc score: 0.940127744973 

KeyboardInterrupt: 

This is very computationnaly intensive, so I will test it later on AWS EC2.